In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
import tensorflow as tf
from pprint import pprint
import cv2

2022-11-27 17:13:43.003512: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-27 17:13:43.003624: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
ROOT_DIR = os.path.abspath('.')
MODELS_DIR = os.path.join(ROOT_DIR, 'models')
CUSTOM_MODEL = os.path.join(MODELS_DIR, 'experiments', 'full_integer.tflite')
# IMAGE_DIR =  os.path.join(ROOT_DIR, 'custom_dataset')
IMAGE_DIR =  os.path.join(ROOT_DIR, 'test')


In [3]:
interpreter = tf.lite.Interpreter(CUSTOM_MODEL)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
pprint(input_details)
pprint(output_details)

{'dtype': <class 'numpy.float32'>,
 'index': 0,
 'name': 'serving_default_input:0',
 'quantization': (0.0, 0),
 'quantization_parameters': {'quantized_dimension': 0,
                             'scales': array([], dtype=float32),
                             'zero_points': array([], dtype=int32)},
 'shape': array([  1, 320, 320,   3], dtype=int32),
 'shape_signature': array([  1, 320, 320,   3], dtype=int32),
 'sparsity_parameters': {}}
{'dtype': <class 'numpy.float32'>,
 'index': 383,
 'name': 'StatefulPartitionedCall:1',
 'quantization': (0.0, 0),
 'quantization_parameters': {'quantized_dimension': 0,
                             'scales': array([], dtype=float32),
                             'zero_points': array([], dtype=int32)},
 'shape': array([ 1, 10], dtype=int32),
 'shape_signature': array([ 1, 10], dtype=int32),
 'sparsity_parameters': {}}


In [4]:
img = cv2.imread(os.path.join(IMAGE_DIR, 'DSQ-6618 (2).jpg'))
_, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']
img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (input_width, input_height))
test_image = np.array(img)
test_image

array([[[ 92,  88,  86],
        [197, 184, 170],
        [252, 235, 212],
        ...,
        [179, 193, 193],
        [179, 191, 190],
        [173, 183, 182]],

       [[ 71,  68,  66],
        [187, 174, 159],
        [252, 236, 212],
        ...,
        [178, 190, 190],
        [179, 189, 188],
        [176, 187, 183]],

       [[ 74,  70,  68],
        [186, 173, 159],
        [249, 231, 207],
        ...,
        [177, 189, 188],
        [176, 185, 183],
        [175, 184, 181]],

       ...,

       [[ 93,  93,  93],
        [ 73,  73,  73],
        [ 99,  99,  99],
        ...,
        [162, 141, 119],
        [185, 159, 136],
        [180, 152, 128]],

       [[105, 105, 105],
        [ 64,  64,  64],
        [ 96,  96,  96],
        ...,
        [161, 140, 119],
        [172, 148, 125],
        [199, 171, 147]],

       [[ 72,  72,  72],
        [ 99,  99,  99],
        [ 87,  87,  87],
        ...,
        [143, 122, 101],
        [157, 134, 110],
        [169, 143, 118]]

In [5]:
def set_input_tensor(interpreter, image):
    """Sets the input tensor."""
    tensor_index = interpreter.get_input_details()[0]['index']
    input_tensor = interpreter.tensor(tensor_index)()[0]
    input_tensor[:, :] = np.expand_dims((image-255)/255, axis=0)


def get_output_tensor(interpreter, index):
    """Returns the output tensor at the given index."""
    output_details = interpreter.get_output_details()[index]
    tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
    return tensor


set_input_tensor(interpreter, test_image)
interpreter.invoke()
boxes = get_output_tensor(interpreter, 1)
classes = get_output_tensor(interpreter, 3)
scores = get_output_tensor(interpreter, 0)
count = int(get_output_tensor(interpreter, 2))

scores[scores.argmax()]

0.8046875